In [1]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Python_Scripts/Assignments/vertisystem/")
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Python_Scripts/Assignments/vertisystem


In [2]:
"""
Flight Data Generator and Analyzer

This script generates synthetic flight data and performs analysis
It creates a large no. of JSON files containing randomized flight records,
then analyzes these files to extract various statistics about the flights

Key features:
1. Generation of synthetic flight data
2. Multithreaded file generation for improved performance
3. Parallel processing for file analysis
4. Analysis of generated data, including:
   - Total no. of records and "miss" (incomplete) records
   - Average and 95th percentile flight durations for top destinations
   - Cities with maximum passenger arrivals and departures

"""

import os
import json
import random
import datetime
import shutil
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time
import multiprocessing
from typing import Dict, List, Tuple, Union

# Constants
N = 5000  # Number of JSON files to generate
M_MIN, M_MAX = 50, 100  # Range for number of flight records per file
K = 150  # Number of cities
L = 0.007  # Probability for NULL values
OUTPUT_DIR = os.path.join(os.getcwd(), "tmp", "flights_1")


# delete it if the directory already exists , checking this to ensure you generate a new sequence for testing
if os.path.exists(OUTPUT_DIR):
  shutil.rmtree(OUTPUT_DIR)
# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
print(OUTPUT_DIR)

/content/drive/MyDrive/Colab Notebooks/Python_Scripts/Assignments/vertisystem/tmp/flights_1


In [4]:
# Generate a list of city names
cities = [f"City_{i}" for i in range(K)]

def generate_flight_record() -> Dict[str, Union[str, int, None]]:
    """
    Generate a single flight record with random data.

    This function creates a dictionary representing a flight record with the following fields:
    - date: ISO format date string (YYYY-MM-DD)
    - origin_city: Randomly selected city name from the 'cities' list
    - destination_city: Randomly selected city name (different from origin) from the 'cities' list
    - flight_duration_secs: Random integer between 1800 and 36000 (30 minutes to 10 hours)
    - passengers_on_board: Random integer between 50 and 300

    There's a small probability (L) that one of the fields will be set to None to simulate incomplete data

    Returns:
        Dict[str, Union[str, int, None]]: A dictionary containing the following key-value pairs:
            - 'date': str
            - 'origin_city': str
            - 'destination_city': str
            - 'flight_duration_secs': int
            - 'passengers_on_board': int
            Any of these values may be None with probability L.

    Example:
        >>> record = generate_flight_record()
        >>> print(record)
        {'date': '2024-06-15', 'origin_city': 'City_42', 'destination_city': 'City_78',
         'flight_duration_secs': 7200, 'passengers_on_board': 180}
    """
    origin, destination = random.sample(cities, 2)
    date = datetime.date(2024, random.randint(1, 12), random.randint(1, 28))
    duration = random.randint(1800, 36000)  # 30 minutes to 10 hours
    passengers = random.randint(50, 300)

    record = {
        "date": date.isoformat(),
        "origin_city": origin,
        "destination_city": destination,
        "flight_duration_secs": duration,
        "passengers_on_board": passengers
    }

    # Introduce NULL values with probability L
    if random.random() < L:
        key = random.choice(list(record.keys()))
        record[key] = None

    return record

def generate_json_chunk(chunk_size: int) -> List[Dict[str, Union[str, int, None]]]:
    """
    Generate a chunk of flight records.

    This function creates a list of flight records by calling the generate_flight_record()
    function multiple times
    It's used as part of the divide-and-conquer approach for generating large datasets

    Args:
        chunk_size (int): The no. of flight records to generate in this chunk.

    Returns:
        List[Dict[str, Union[str, int, None]]]: A list of dictionaries, where each dictionary
        represents a flight record with the structure defined in generate_flight_record()

    Raises:
        AssertionError: If chunk_size is not a positive integer

    Example:
        >>> chunk = generate_json_chunk(5)
        >>> print(len(chunk))
        5
        >>> print(chunk[0].keys())
        dict_keys(['date', 'origin_city', 'destination_city', 'flight_duration_secs', 'passengers_on_board'])
    """
    assert isinstance(chunk_size, int) and chunk_size > 0, "chunk_size must be a positive integer"
    return [generate_flight_record() for _ in range(chunk_size)]

def generate_json_file(file_number: int) -> None:
    """
    Generate a JSON file with flight records.

    This function creates a single JSON file containing a random no. of flight records
    (between M_MIN and M_MAX). The file is named using the format:
    "MM-YY-OriginCity-flights-FileNumber.json"

    Args:
        file_number (int): The no. of the file being generated, used in the filename

    Raises:
        AssertionError: If file_number is not a non-negative integer
        IOError: If there's an error writing to the file

    Example:
        >>> generate_json_file(42)
        # Creates a file like "08-24-City_56-flights-42.json" in the OUTPUT_DIR
    """
    assert isinstance(file_number, int) and file_number >= 0, "file_number must be a non-negative integer"

    month = random.randint(1, 12)
    year = 24
    origin_city = random.choice(cities)
    filename = f"{OUTPUT_DIR}/{month:02d}-{year:02d}-{origin_city}-flights-{file_number}.json"

    chunk_size = random.randint(M_MIN, M_MAX)
    records = generate_json_chunk(chunk_size)

    try:
        with open(filename, 'w') as f:
            json.dump(records, f)
    except IOError as e:
        print(f"Error writing to file {filename}: {e}")
        raise

def process_file(filename: str) -> Dict[str, Union[int, Dict]]:
    """
    Process a single file and return its statistics

    This function reads a JSON file of flight records and computes various statistics:
    - Total no. of records
    - No. of "miss" (incomplete) records
    - Flight durations for each destination
    - Passenger counts for arrivals and departures for each city

    Args:
        filename (str): The name of the file to process

    Returns:
        Dict[str, Union[int, Dict]]: A dictionary containing the following key-value pairs:
            - 'total_records': int
            - 'miss_records': int
            - 'flight_durations': Dict[str, List[int]]
            - 'city_passengers': Dict[str, Dict[str, int]]

    Raises:
        AssertionError: If filename is not a string.
        FileNotFoundError: If the specified file does not exist.
        json.JSONDecodeError: If the file contains invalid JSON.

    Example:
        >>> stats = process_file('08-24-City_56-flights-42.json')
        >>> print(stats['total_records'])
        75
    """
    assert isinstance(filename, str), "filename must be a string"

    try:
        with open(os.path.join(OUTPUT_DIR, filename), 'r') as f:
            records = json.load(f)
    except FileNotFoundError:
        print(f"File not found: {filename}")
        raise
    except json.JSONDecodeError:
        print(f"Invalid JSON in file: {filename}")
        raise

    file_stats = {
        'total_records': len(records),
        'miss_records': 0,
        'flight_durations': {},
        'city_passengers': {city: {"arrived": 0, "left": 0} for city in cities}
    }

    for record in records:
        if None in record.values():
            file_stats['miss_records'] += 1
        else:
            dest = record["destination_city"]
            if dest not in file_stats['flight_durations']:
                file_stats['flight_durations'][dest] = []
            file_stats['flight_durations'][dest].append(record["flight_duration_secs"])

            file_stats['city_passengers'][record["origin_city"]]["left"] += record["passengers_on_board"]
            file_stats['city_passengers'][record["destination_city"]]["arrived"] += record["passengers_on_board"]

    return file_stats

def merge_stats(stats_list: List[Dict[str, Union[int, Dict]]]) -> Dict[str, Union[int, Dict]]:
    """
    Merge statistics from multiple files.

    This function takes a list of statistics dictionaries (one for each processed file)
    and merges them into a single dictionary of combined statistics.

    Args:
        stats_list (List[Dict[str, Union[int, Dict]]]): A list of statistics dictionaries from processed files.

    Returns:
        Dict[str, Union[int, Dict]]: A dictionary containing the following key-value pairs:
            - 'total_records': int
            - 'miss_records': int
            - 'flight_durations': Dict[str, List[int]]
            - 'city_passengers': Dict[str, Dict[str, int]]

    Raises:
        AssertionError: If stats_list is not a list or if any item in the list is not a dictionary.

    Example:
        >>> stats1 = {'total_records': 100, 'miss_records': 5, ...}
        >>> stats2 = {'total_records': 150, 'miss_records': 7, ...}
        >>> merged = merge_stats([stats1, stats2])
        >>> print(merged['total_records'])
        250
    """
    assert isinstance(stats_list, list), "stats_list must be a list"
    assert all(isinstance(stats, dict) for stats in stats_list), "All items in stats_list must be dictionaries"

    merged_stats = {
        'total_records': 0,
        'miss_records': 0,
        'flight_durations': {},
        'city_passengers': {city: {"arrived": 0, "left": 0} for city in cities}
    }

    for stats in stats_list:
        merged_stats['total_records'] += stats['total_records']
        merged_stats['miss_records'] += stats['miss_records']

        for dest, durations in stats['flight_durations'].items():
            if dest not in merged_stats['flight_durations']:
                merged_stats['flight_durations'][dest] = []
            merged_stats['flight_durations'][dest].extend(durations)

        for city, passengers in stats['city_passengers'].items():
            merged_stats['city_passengers'][city]["arrived"] += passengers["arrived"]
            merged_stats['city_passengers'][city]["left"] += passengers["left"]

    return merged_stats

def analyze_files() -> Tuple[int, int, Dict[str, Dict[str, float]], Tuple[str, Dict[str, int]], Tuple[str, Dict[str, int]]]:
    """
    Analyze the generated JSON files using parallel processing.

    This function reads all JSON files in the OUTPUT_DIR, processes them in parallel,
    and computes various statistics about the flight data.

    Returns:
        Tuple[int, int, Dict[str, Dict[str, float]], Tuple[str, Dict[str, int]], Tuple[str, Dict[str, int]]]:
            - total_records (int): Total number of records processed across all files.
            - miss_records (int): Total number of incomplete records across all files.
            - stats (Dict[str, Dict[str, float]]): Dictionary of flight duration stats for top 25 destinations.
              Each destination has 'avg' and 'p95' keys with float values.
            - max_arrived (Tuple[str, Dict[str, int]]): Tuple containing the city name (str) with max arrived passengers
              and a dictionary with 'arrived' and 'left' passenger counts (int).
            - max_left (Tuple[str, Dict[str, int]]): Tuple containing the city name (str) with max departed passengers
              and a dictionary with 'arrived' and 'left' passenger counts (int).

    Raises:
        FileNotFoundError: If the OUTPUT_DIR does not exist or is empty.

    Example:
        >>> total, miss, stats, max_arr, max_dep = analyze_files()
        >>> print(f"Total records: {total}, miss records: {miss}")
        Total records: 375000, miss records: 2625
        >>> print(stats['City_1'])
        {'avg': 18360.25, 'p95': 32400.00}
        >>> print(max_arr)
        ('City_78', {'arrived': 1500000, 'left': 1480000})
    """
    filenames = os.listdir(OUTPUT_DIR)
    if not filenames:
        raise FileNotFoundError(f"No files found in {OUTPUT_DIR}")

    # Use ProcessPoolExecutor for parallel file processing
    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        stats_list = list(executor.map(process_file, filenames))

    # Merge the results
    merged_stats = merge_stats(stats_list)

    total_records = merged_stats['total_records']
    miss_records = merged_stats['miss_records']

    # Calculate AVG and P95 for top 25 destination cities
    flight_durations = merged_stats['flight_durations']
    top_25_destinations = sorted(flight_durations.items(), key=lambda x: len(x[1]), reverse=True)[:25]
    stats = {}
    for dest, durations in top_25_destinations:
        avg = sum(durations) / len(durations)
        p95 = sorted(durations)[int(len(durations) * 0.95)]
        stats[dest] = {"avg": avg, "p95": p95}

    # Find cities with max passengers arrived and left
    city_passengers = merged_stats['city_passengers']
    max_arrived = max(city_passengers.items(), key=lambda x: x[1]["arrived"])
    max_left = max(city_passengers.items(), key=lambda x: x[1]["left"])

    return total_records, miss_records, stats, max_arrived, max_left


def main() -> None:
        """
        Main function to run the entire data generation and analysis pipeline.

        This function orchestrates the entire process:
        1. Generates N JSON files containing flight records using multi-threading
        2. Analyzes the generated files to extract statistics using parallel processing
        3. Prints the results of the analysis

        The function also measures and reports the total execution time.

        Returns:
            None

        Raises:
            Exception: If any error occurs during the execution of the pipeline.

        Example:
            >>> main()
            Generating 5000 JSON files...
            Analyzing files...
            Total records processed: 375000
            miss records: 2625
            Total run duration: 45.23 seconds
            ...
        """
        start_time = time.time()

        try:
            # Generation phase
            print(f"Generating {N} JSON files...")
            with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count() * 2) as executor:
                list(executor.map(generate_json_file, range(N)))

            # Analysis phase
            print("Analyzing files...")
            total_records, miss_records, flight_stats, max_arrived, max_left = analyze_files()

            end_time = time.time()
            total_duration = end_time - start_time

            # Print results
            print(f"Total records processed: {total_records}")
            print(f"miss records: {miss_records}")
            print(f"Total run duration: {total_duration:.2f} seconds")

            print("\nAVG and P95 of flight duration for Top 25 destination cities:")
            for dest, stats in flight_stats.items():
                print(f"{dest}: AVG = {stats['avg']:.2f} seconds, P95 = {stats['p95']:.2f} seconds")

            print(f"\nCity with max passengers arrived: {max_arrived[0]} ({max_arrived[1]['arrived']} passengers)")
            print(f"City with max passengers left: {max_left[0]} ({max_left[1]['left']} passengers)")

        except Exception as e:
            print(f"An error occurred during execution: {str(e)}")
            raise

if __name__ == "__main__":
        main()

Generating 5000 JSON files...
Analyzing files...
Total records processed: 374852
miss records: 2648
Total run duration: 104.58 seconds

AVG and P95 of flight duration for Top 25 destination cities:
City_25: AVG = 18729.73 seconds, P95 = 34294.00 seconds
City_90: AVG = 18986.49 seconds, P95 = 34389.00 seconds
City_136: AVG = 18939.40 seconds, P95 = 34428.00 seconds
City_109: AVG = 19074.32 seconds, P95 = 34222.00 seconds
City_35: AVG = 19011.84 seconds, P95 = 34261.00 seconds
City_63: AVG = 19200.48 seconds, P95 = 34553.00 seconds
City_27: AVG = 18839.53 seconds, P95 = 34235.00 seconds
City_75: AVG = 18941.06 seconds, P95 = 34151.00 seconds
City_16: AVG = 18656.44 seconds, P95 = 34293.00 seconds
City_8: AVG = 18775.65 seconds, P95 = 34236.00 seconds
City_21: AVG = 19014.55 seconds, P95 = 34336.00 seconds
City_60: AVG = 18815.46 seconds, P95 = 34212.00 seconds
City_79: AVG = 19101.40 seconds, P95 = 34322.00 seconds
City_68: AVG = 18744.95 seconds, P95 = 34246.00 seconds
City_18: AVG = 19